In [42]:
# importing the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# importing the datasets
training_data  = pd.read_csv('data/Train_data.csv')
test_data = pd.read_csv('data/Test_data.csv')

## Preprocessing

In [43]:
# checking the shape of the datasets
print(training_data.shape)
print(test_data.shape)

(25192, 42)
(22544, 41)


In [44]:
# checking the null values of the datasets
print(training_data.isnull().sum())
print('------------------')
print(test_data.isnull().sum())

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [48]:
# checking the unique values of the datasets
print(training_data.nunique())

duration                        758
protocol_type                     3
service                          66
flag                             11
src_bytes                      1665
dst_bytes                      3922
land                              2
wrong_fragment                    3
urgent                            2
hot                              22
num_failed_logins                 5
logged_in                         2
num_compromised                  28
root_shell                        2
su_attempted                      3
num_root                         28
num_file_creations               20
num_shells                        2
num_access_files                  7
num_outbound_cmds                 1
is_host_login                     1
is_guest_login                    2
count                           466
srv_count                       414
serror_rate                      70
srv_serror_rate                  56
rerror_rate                      72
srv_rerror_rate             

In [49]:
# normalizing the dataset


ValueError: could not convert string to float: 'tcp'

## Exploratory Data Analysis

## Feature selection

## Model training

## Model evaluation